# Biostat 257 Homework 4

**Due May 20 @ 11:59PM**

In [1]:
versioninfo()

Julia Version 1.7.1
Commit ac5cc99908 (2021-12-22 19:35 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.5.0)
  CPU: Intel(R) Core(TM) i5-8279U CPU @ 2.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, skylake)


We are going to try different numerical methods learnt in class on the [Google PageRank problem](https://en.wikipedia.org/wiki/PageRank).

## Q1 (5 pts) Recognize structure

Let $\mathbf{A} \in \{0,1\}^{n \times n}$ be the connectivity matrix of $n$ web pages with entries
$$
\begin{eqnarray*}
	a_{ij}= \begin{cases}
	1 & \text{if page $i$ links to page $j$} \\
	0 & \text{otherwise}
	\end{cases}.
\end{eqnarray*}
$$
$r_i = \sum_j a_{ij}$ is the out-degree of page $i$. That is $r_i$ is the number of links on page $i$. Imagine a random surfer exploring the space of $n$ pages according to the following rules.  

- From a page $i$ with $r_i>0$
    * with probability $p$, (s)he randomly chooses a link on page $i$ (uniformly) and follows that link to the next page  
    * with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
- From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
    
The process defines a Markov chain on the space of $n$ pages. Write the transition matrix $\mathbf{P}$ of the Markov chain as a sparse matrix plus rank 1 matrix.

### Answer

Since $A$ is a binary matrix with links and $r_{i} = \sum_{j}a_{ij}$ is the out-degree of page $i$, assuming $r_{i} > 0$ for all $i$, a natural transition probability based on $A$ is $\mathrm{diag}(1/r_{i})A$, where we scale the columns of $A$ by their respective out-degrees. Accounting for $p$, we have $p\mathrm{diag}(1/r_{i})A + (1 - p)\frac{1}{n}1_{n}1_{n}^{T}$. Next, let $z_{i} = 1(r_{i} = 0)$. Next, set $r_{i}^{*} = \infty$ if $r_{i} = 0$ and $r_{i}^{*} = r_{i}$ otherwise, so that $1/r_{i}^{*} = 0$ for $z_{i} = 1$. (Alternatively, letting $r$ be the vector of $r_{i}$ values and $z$ be the vector of $z_{i}$ values, $1/r^{*} = 1/r \odot (1-z)$, where $\odot$ is the entrywise Hadamard product.) Our final $P$ follows:

\begin{eqnarray*}
P &=& p\mathrm{diag}(1/r_{i}^{*})A + (1 - p)\frac{1}{n}1_{n}1_{n}^{T} + p\frac{1}{n}\mathrm{diag}(z_{i})1_{n}1_{n}^{T}\\
 &=& p\mathrm{diag}(1/r_{i}^{*})A + \frac{1}{n}((1 - p)I + p\mathrm{diag}(z_{i}))1_{n}1_{n}^{T}\\
\end{eqnarray*}

## Q2 Relate to numerical linear algebra

According to standard Markov chain theory, the (random) position of the surfer converges to the stationary distribution $\mathbf{x} = (x_1,\ldots,x_n)^T$ of the Markov chain. $x_i$ has the natural interpretation of the proportion of times the surfer visits page $i$ in the long run. Therefore $\mathbf{x}$ serves as page ranks: a higher $x_i$ means page $i$ is more visited. It is well-known that $\mathbf{x}$ is the left eigenvector corresponding to the top eigenvalue 1 of the transition matrix $\mathbf{P}$. That is $\mathbf{P}^T \mathbf{x} = \mathbf{x}$. Therefore $\mathbf{x}$ can be solved as an **eigen-problem**. It can also be cast as **solving a linear system**. Since the row sums of $\mathbf{P}$ are 1, $\mathbf{P}$ is rank deficient. We can replace the first equation by the $\sum_{i=1}^n x_i = 1$.

Hint: For iterative solvers, we don't need to replace the 1st equation. We can use the matrix $\mathbf{I} - \mathbf{P}^T$ directly if we start with a vector with all positive entries.

## Q3 (10 pts) Explore data

Obtain the connectivity matrix `A` from the `SNAP/web-Google` data in the MatrixDepot package. 

In [2]:
using MatrixDepot

md = mdopen("SNAP/web-Google")
# display documentation for the SNAP/web-Google data
mdinfo(md)

┌ Info: verify download of index files...
└ @ MatrixDepot /Users/danielzhou/.julia/packages/MatrixDepot/GEDc3/src/MatrixDepot.jl:139
┌ Info: reading database
└ @ MatrixDepot /Users/danielzhou/.julia/packages/MatrixDepot/GEDc3/src/download.jl:23
┌ Info: adding metadata...
└ @ MatrixDepot /Users/danielzhou/.julia/packages/MatrixDepot/GEDc3/src/download.jl:67
┌ Info: adding svd data...
└ @ MatrixDepot /Users/danielzhou/.julia/packages/MatrixDepot/GEDc3/src/download.jl:69
┌ Info: writing database
└ @ MatrixDepot /Users/danielzhou/.julia/packages/MatrixDepot/GEDc3/src/download.jl:74
┌ Info: used remote sites are sparse.tamu.edu with MAT index and math.nist.gov with HTML index
└ @ MatrixDepot /Users/danielzhou/.julia/packages/MatrixDepot/GEDc3/src/MatrixDepot.jl:141


# SNAP/web-Google

###### MatrixMarket matrix coordinate pattern general

---

  * UF Sparse Matrix Collection, Tim Davis
  * http://www.cise.ufl.edu/research/sparse/matrices/SNAP/web-Google
  * name: SNAP/web-Google
  * [Web graph from Google]
  * id: 2301
  * date: 2002
  * author: Google
  * ed: J. Leskovec
  * fields: name title A id date author ed kind notes
  * kind: directed graph

---

  * notes:
  * Networks from SNAP (Stanford Network Analysis Platform) Network Data Sets,
  * Jure Leskovec http://snap.stanford.edu/data/index.html
  * email jure at cs.stanford.edu
  * 
  * Google web graph
  * 
  * Dataset information
  * 
  * Nodes represent web pages and directed edges represent hyperlinks between them.
  * The data was released in 2002 by Google as a part of Google Programming
  * Contest.
  * 
  * Dataset statistics
  * Nodes   875713
  * Edges   5105039
  * Nodes in largest WCC    855802 (0.977)
  * Edges in largest WCC    5066842 (0.993)
  * Nodes in largest SCC    434818 (0.497)
  * Edges in largest SCC    3419124 (0.670)
  * Average clustering coefficient  0.6047
  * Number of triangles     13391903
  * Fraction of closed triangles    0.05523
  * Diameter (longest shortest path)    22
  * 90-percentile effective diameter    8.1
  * 
  * Source (citation)
  * 
  * J. Leskovec, K. Lang, A. Dasgupta, M. Mahoney. Community Structure in Large
  * Networks: Natural Cluster Sizes and the Absence of Large Well-Defined Clusters.
  * arXiv.org:0810.1355, 2008.
  * 
  * Google programming contest, 2002
  * http://www.google.com/programming-contest/
  * 
  * Files
  * File    Description
  * web-Google.txt.gz   Webgraph from the Google programming contest, 2002

---

916428 916428 5105039


In [3]:
# connectivity matrix
A = md.A

916428×916428 SparseArrays.SparseMatrixCSC{Bool, Int64} with 5105039 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿

Compute summary statistics:
* How much memory does `A` take? If converted to a `Matrix{Float64}` (don't do it!), how much memory will it take?  
* number of web pages
* number of edges (web links). 
* number of dangling nodes (pages with no out links)
* histogram of in-degrees  
* list the top 20 pages with the largest in-degrees?  
* histogram of out-degrees
* which the top 20 pages with the largest out-degrees?
* visualize the sparsity pattern of $\mathbf{A}$ or a submatrix of $\mathbf{A}$ say `A[1:10000, 1:10000]`. 

**Hint**: For plots, you can use the [UnicodePlots.jl](https://github.com/Evizero/UnicodePlots.jl) package (`spy`, `histogram`, etc), which is fast for large data. 

### Answer

In [4]:
@show typeof(A)

typeof(A) = SparseArrays.SparseMatrixCSC{Bool, Int64}


SparseArrays.SparseMatrixCSC{Bool, Int64}

Since $A$ is a boolean matrix, $A$ takes $5,105,039\times 2 = 10,210,078$ bytes, or about $9.74$ MB. Since $A$ is $916,428\times 916,428$, a dense $A$ would take $916,428^2 \times 2$ bytes, or about $1.53$ TB.

Due to information on the structure of $A$ given at the beginning of this problem, there are $916,428$ web pages.

Taking the number of web links to be the number of nonzero entries in $A$, we have $5,105,039$ links and $916,428^2 - 5,105,039 = 839,835,174,145$ pages with no out links.

We compute the additional statistics:

In-degrees:

In [5]:
using UnicodePlots

In [6]:
n = size(A, 1)
i_array = zeros(n)
i_array .= sum(A, dims = 2)

#for i in 1:n
#    i_array[i] = sum(A[:, i])
#    o_array[i] = sum(A[i, :])
#end

histogram(i_array)

                  ┌                                        ┐ 
   [  0.0,  20.0) ┤████████████████████████████████  891798  
   [ 20.0,  40.0) ┤▊ 22628                                   
   [ 40.0,  60.0) ┤▏ 1329                                    
   [ 60.0,  80.0) ┤▏ 371                                     
   [ 80.0, 100.0) ┤▏ 124                                     
   [100.0, 120.0) ┤▏ 86                                      
   [120.0, 140.0) ┤▏ 29                                      
   [140.0, 160.0) ┤▏ 15                                      
   [160.0, 180.0) ┤▏ 15                                      
   [180.0, 200.0) ┤▏ 8                                       
   [200.0, 220.0) ┤▏ 7                                       
   [220.0, 240.0) ┤▏ 3                                       
   [240.0, 260.0) ┤▏ 8                                       
   [260.0, 280.0) ┤▏ 3                                       
   [280.0, 300.0) ┤  0                                       
   [300.

In [7]:
### Top 20 in-degree pages
in_degree_top_20 = Vector{Int64}(undef, 20)
for i in 1:20
    m, ix = findmax(i_array)
    println("Maximum ", i, "th in-degree count ", m, " at index ", ix, ".")
    in_degree_top_20[i] = ix
    deleteat!(i_array, ix)
end

Maximum 1th in-degree count 456.0 at index 506743.
Maximum 2th in-degree count 372.0 at index 203749.
Maximum 3th in-degree count 372.0 at index 305229.
Maximum 4th in-degree count 330.0 at index 768089.
Maximum 5th in-degree count 277.0 at index 808640.
Maximum 6th in-degree count 268.0 at index 412409.
Maximum 7th in-degree count 265.0 at index 600476.
Maximum 8th in-degree count 258.0 at index 376427.
Maximum 9th in-degree count 257.0 at index 156951.
Maximum 10th in-degree count 256.0 at index 885720.
Maximum 11th in-degree count 253.0 at index 667578.
Maximum 12th in-degree count 248.0 at index 685688.
Maximum 13th in-degree count 247.0 at index 282139.
Maximum 14th in-degree count 245.0 at index 598182.
Maximum 15th in-degree count 244.0 at index 579308.
Maximum 16th in-degree count 231.0 at index 411589.
Maximum 17th in-degree count 229.0 at index 321088.
Maximum 18th in-degree count 225.0 at index 838263.
Maximum 19th in-degree count 216.0 at index 302731.
Maximum 20th in-degre

Out-degrees:

In [8]:
### Top 20 out-degree pages

o_array = zeros(n)
o_array .= vec(sum(A, dims = 1))

histogram(o_array)

                    ┌                                        ┐ 
   [   0.0,  500.0) ┤████████████████████████████████  916114  
   [ 500.0, 1000.0) ┤▏ 180                                     
   [1000.0, 1500.0) ┤▏ 32                                      
   [1500.0, 2000.0) ┤▏ 20                                      
   [2000.0, 2500.0) ┤▏ 16                                      
   [2500.0, 3000.0) ┤▏ 20                                      
   [3000.0, 3500.0) ┤▏ 18                                      
   [3500.0, 4000.0) ┤▏ 10                                      
   [4000.0, 4500.0) ┤▏ 9                                       
   [4500.0, 5000.0) ┤▏ 4                                       
   [5000.0, 5500.0) ┤▏ 4                                       
   [5500.0, 6000.0) ┤  0                                       
   [6000.0, 6500.0) ┤▏ 1                                       
                    └                                        ┘ 
                                     Fre

In [9]:
out_degree_top_20 = Vector{Int64}(undef, 20)
for i in 1:20
    m, ix = findmax(o_array)
    println("Maximum ", i, "th out-degree count ", m, " at index ", ix, ".")
    out_degree_top_20[i] = ix
    deleteat!(o_array, ix)
end

Maximum 1th out-degree count 6326.0 at index 537040.
Maximum 2th out-degree count 5354.0 at index 597621.
Maximum 3th out-degree count 5271.0 at index 504141.
Maximum 4th out-degree count 5182.0 at index 751382.
Maximum 5th out-degree count 5097.0 at index 32164.
Maximum 6th out-degree count 4847.0 at index 885601.
Maximum 7th out-degree count 4731.0 at index 163075.
Maximum 8th out-degree count 4620.0 at index 819218.
Maximum 9th out-degree count 4550.0 at index 605852.
Maximum 10th out-degree count 4484.0 at index 828956.
Maximum 11th out-degree count 4220.0 at index 551826.
Maximum 12th out-degree count 4219.0 at index 41909.
Maximum 13th out-degree count 4206.0 at index 558786.
Maximum 14th out-degree count 4187.0 at index 459072.
Maximum 15th out-degree count 4180.0 at index 407608.
Maximum 16th out-degree count 4084.0 at index 213430.
Maximum 17th out-degree count 4015.0 at index 765322.
Maximum 18th out-degree count 4010.0 at index 384663.
Maximum 19th out-degree count 3988.0 at

Visualization

In [10]:
n_v = 10_000
spy(A[1:n_v, 1:n_v])

         ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀Sparsity Pattern⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀    
         ┌──────────────────────────────────────────┐    
       1 │⠀⠀⠀⠀⠀⠀⠀⠂⠀⠀⠀⠀⢀⢂⡂⠐⠀⠀⠀⠠⠀⠀⠀⠔⠄⠀⠈⢀⠀⠀⠉⠀⠀⠁⠀⠀⠐⠀⠄⠀⠐⠀│ > 0
         │⠀⠀⠀⠀⠀⠂⡂⠀⠄⠄⠈⠈⡔⠀⠀⠁⠂⠀⠀⠀⠀⠠⠁⠒⠁⠀⠄⠀⠐⠀⠈⠀⠀⡐⠀⠀⠠⠀⡀⠀⠉⠀│ < 0
         │⠀⠐⠠⠀⠀⠁⢐⠄⠀⠢⠠⠀⠀⡀⠈⠀⠋⠀⠀⠀⠀⠀⠂⡀⠀⠈⠄⠀⠄⠁⠈⠀⠠⠀⠰⠀⠀⠀⠀⢀⠆⠀│    
         │⠀⠈⠠⠂⠀⠈⠀⡠⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠈⠄⠁⢀⡀⠐⠄⠄⠀⠀⢀⠀⠈⠂⢠⠀⠢⠐⠀⠀⠀⡀│    
         │⠀⠀⠐⡁⠀⡀⠀⠀⠠⠀⠁⠀⠀⠀⡄⠀⠈⠁⠀⠀⠀⠀⠠⠂⠀⠀⠀⠀⠀⠂⠪⠀⠀⠂⠀⠀⠀⠄⢀⠀⠁⠀│    
         │⠂⢀⠀⠁⠀⠀⡀⡉⠀⠀⠠⠂⠐⢀⠄⠁⠐⠀⠈⠀⠀⠀⠀⠀⡀⠐⠁⡠⠀⠂⠚⠀⠁⠀⠐⢀⠆⠀⠈⠄⢀⠂│    
         │⠀⠀⠀⠁⠀⡄⠀⠀⠀⠀⠒⠀⢄⠄⢂⠀⠂⠀⠀⠄⠀⢀⠀⠣⠀⢀⠀⠀⠀⠀⠠⡀⠀⠀⢠⠄⠀⡀⠀⠂⠂⠀│    
         │⢀⠈⠀⠀⠂⠀⠀⠁⢠⠈⠄⠁⠀⠀⡈⠈⠠⠀⢀⡦⠀⠀⠅⠐⠀⠨⠁⠨⠀⠄⠠⢀⠀⠐⢐⠀⠀⠉⠀⠀⠠⠉│    
         │⠀⠀⠨⠀⠋⠀⠄⠀⠂⠀⠈⠀⠀⠀⠈⠀⠀⠀⠀⡑⠀⢀⠈⠈⠀⠀⠀⠈⠀⠂⠐⠼⠀⠠⠀⠀⡈⠁⠀⠀⠡⠄│    
         │⠠⡁⠀⠀⠀⠀⠀⠀⠀⠀⠂⢀⠀⠀⠀⡄⠀⠀⠀⠤⠀⠀⠁⠀⡂⠀⠀⠀⠄⠀⢈⠀⢔⡊⣨⠈⠂⠀⠄⢀⠀⠀│    
         │⠀⠂⠀⠈⠀⠀⠀⠀⠀⠀⠈⠀⡀⠀⠂⠠⠐⠀⠄⠀⠀⠀⠀⠀⠀⠤⠀⠀⠀⠠⠘⠐⠀⠄⢘⠀⠀⢐⠄⠀⠁⠀│    
         │⢀⠄⢠⠄⡊⠂⠢⢀⠀⠀⠀⠀⠄⡀⡑⠀⡐⠀⠁⡂⠀⠀⠀⠰⠂⢠⠄⠈⠀⠈⠐⠀⠀⠀⠉⠀⠀⡀⠀⠀⠃⠀│    
         │⠀⠀⠅⠀⡁⠀⠀⠀⠀⠀⢁⢀⠀⡀⡀⡐⠀⠀⠈⠈⠔⠄⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠄⢉⠠⠁⠀⠁⠀⠀⠀│    
         │⠊⢈⠀⠁⠀⠀⠀⠅⠀⠂⠀⡀⠀⠀⠀⠀⡀⠀⠀⠐⠄⠁⠠⠀⠐⠈⠀⠀⠀⡀⢠⠀⠀⠀⢰⠀⠀⠀⢀⠀⠠⠂│    
         │⠈⠀⠀⠈⠄⠀⠀⠄⠐⠀⠠⠁⠁⠀⡂⠀⠠⠄⠌⠅⣀⠀⠀⢀⠀⠠⠀⠠⠠⠀⠲⢂⠀⠰⡄⠀⢀⠬⠈⠀⠀⠀│    
         │⠈⠀⠀⠄

Note that finding the top 20 pages with in- and out-degree counts was done with a naive for-loop, because sorting the respective arrays would have taken $O(n\log n)$, whereas a naive search would have taken $20 n$, especially since $\log n \gg 20$.

## Q4 (5 pts) Dense linear algebra? 

Consider the following methods to obtain the page ranks of the `SNAP/web-Google` data. 

1. A dense linear system solver such as LU decomposition.  
2. A dense eigen-solver for asymmetric matrix.  

For the LU approach, estimate (1) the memory usage and (2) how long it will take assuming that the LAPACK functions can achieve the theoretical throughput of your computer. 

### Answer

Since we are using dense linear system solvers on a sparse system, the worst-case memory usage would be approximately the number of nodes times $8$ bytes per node for storage, or about $1.53\text{ TB} / 2 \times 8 \approx 6.11$ TB. Per Dr. Zhou's notes, LU decomposition takes $\frac{2}{3}n^{3} + 2n^{2}$ flops.

The theoretical flops per second for my computer would be (4 cores)$\times$(2.4 GHz)$\times$(8 flops (one per thread?))$\approx$ $7.68\times 10^{10}$ flops/s at peak performance. Since $n = 916,428$, the expected time would be:
\begin{eqnarray*}
T &=& \frac{\frac{2}{3}n^{3} + 2n^{2}}{7.68 \times 10^{10}}\\
 &\approx& 6.68\times 10^{6} \text{ s.} \approx 77.33 \text{ days}
\end{eqnarray*}

Note that this calculation assumes my laptop will be at peak performance througout 77.33 days without the hardware degrading: a tall assumption.

## Q5 (75 pts) Iterative solvers

Set the _teleportation_ parameter at $p = 0.85$. Consider the following methods for solving the PageRank problem. 

1. An iterative linear system solver such as GMRES. 
2. An iterative eigen-solver such as Arnoldi method.

For iterative methods, we have many choices in Julia. See a list of existing Julia packages for linear solvers at this [page](https://jutho.github.io/KrylovKit.jl/stable/#Package-features-and-alternatives-1). The start-up code below uses the [KrylovKit.jl](https://github.com/Jutho/KrylovKit.jl) package. You can use other packages if you prefer. Make sure to utilize the special structure of $\mathbf{P}$ (sparse + rank 1) to speed up the matrix-vector multiplication. 

### Step 1 (15 pts)

Let's implement a type `PageRankImPt` that mimics the matrix $\mathbf{M} = \mathbf{I} - \mathbf{P}^T$. For iterative methods, all we need to provide are methods for evaluating $\mathbf{M} \mathbf{v}$ and $\mathbf{M}^T \mathbf{v}$ for arbitrary vector $\mathbf{v}$.

### Answer

Prior to implementing $M = I - P^{T}$, recall our answer to Q1: $P = p\mathrm{diag}(1/r_{i}^{*})A + \frac{1}{n}1_{n}1_{n}^{T}((1 - p)I + p\mathrm{diag}(z_{i}))$. Then,

\begin{eqnarray*}
Mv &=& v - P^{T}v\\
 &=& v - (pA^{T}\mathrm{diag}(1/r_{i}^{*})v + \frac{1}{n}1_{n}1_{n}^{T}((1 - p)I + p\mathrm{diag}(z_{i}))v)\\
 &=& v - (pA^{T}((1/r_{i}^{*}) \odot v) + (1 - p)1_{n}\bar{v} + p\frac{1}{n}1_{n}1_{n}^{T}(z\odot v))\\
\end{eqnarray*}

where $\bar{v}$ refers to the average of the entries of $v$.

Analogously, we compute $M^{T}v$:

\begin{eqnarray*}
M^{T}v &=& v - Pv\\
 &=& v - (p\mathrm{diag}(1/r_{i}^{*})Av + \frac{1}{n}((1 - p)I + p\mathrm{diag}(z_{i}))1_{n}1_{n}^{T}v)\\
 &=& v - (p\mathrm{diag}(1/r_{i}^{*})Av + (1-p)1_{n}\bar{v} + pz\bar{v}\\
\end{eqnarray*}

where $1/r^{*}$ is the vector of $1/r_{i}^{*}$ values, $z$ is the vector of $z_{i}$ values and $\odot$ refers to the entrywise (Hadamard) product between two vectors.

To get $M_{ij} = I_{ij} - P_{ji}$, we substitute $\phi_{j}$, the $j$th column vector of the $n\times n$ identity matrix into $v$ to get the $j$th column of $M$:
\begin{eqnarray*}
M\phi_{j} &=& \phi_{j} - p(1/r_{j}^{*})A^{T}\phi_{j} - \frac{1-p}{n}1_{n} - \frac{p}{n}z\\
 &=& \phi_{j} - p(1/r_{j}^{*})A_{j,:}^{T} - \frac{1-p}{n}1_{n} - \frac{p}{n}z\\
\end{eqnarray*}

To get the $i,j$th entry, we multiply our result by $\phi_{i}^{T}$:
\begin{eqnarray*}
\phi_{i}^{T}M\phi_{j} &=& \phi_{i}^{T}\phi_{j} - p\phi_{i}^{T}(1/r_{j}^{*})A_{j,:}^{T} - \frac{1}{n^{2}} - \frac{p}{n}z_{i}\\
 &=& \phi_{i}^{T}\phi_{j} - \frac{p}{r_{j}^{*}}\phi_{i}^{T}A_{j,:}^{T} - \frac{1 - p}{n} - \frac{p}{n}z_{i}\\
 &=& \phi_{i}^{T}\phi_{j} - \frac{p}{r_{j}^{*}}a_{ji} - \frac{1 - p}{n} - \frac{p}{n}z_{i}\\
\end{eqnarray*}

In [11]:
using BenchmarkTools, LinearAlgebra, SparseArrays, Revise

# a type for the matrix M = I - P^T in PageRank problem
struct PageRankImPt{TA <: Number, IA <: Integer, T <: AbstractFloat} <: AbstractMatrix{T}
    A         :: SparseMatrixCSC{TA, IA} # adjacency matrix
    telep     :: T
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate. I - P^T
    out_degrees     :: Vector{IA}
    inv_out_degrees :: Vector{T}
    dangling_page   :: SparseVector{Bool, IA}
    storage_n       :: Vector{T}
end

# constructor
function PageRankImPt(A::SparseMatrixCSC, telep::T) where T <: AbstractFloat
    n = size(A, 1)
    # generate a copy
    n = size(A, 1)
    out_degrees = vec(sum(A, dims = 2)) #TODO: 
    dangling_page = sparse(map(d -> (d == 0), out_degrees))
    inv_out_degrees = 1. ./ out_degrees
    inv_out_degrees[dangling_page] .= 0
    storage_n   = Vector{T}(undef, n)
    PageRankImPt(A, telep, out_degrees, inv_out_degrees, dangling_page, storage_n)
end

LinearAlgebra.issymmetric(::PageRankImPt) = false
Base.size(M::PageRankImPt) = size(M.A)
# TODO: implement this function for evaluating M[i, j]
function Base.getindex(M::PageRankImPt, i, j)
    n = size(M)[1]
    m = Int64(i == j) - M.telep * Int64(M.A[j, i]) * M.inv_out_degrees[j] - 
        M.telep / n * Int64(M.dangling_page[i]) - (1. - M.telep) / n
    return m
end

# overwrite `out` by `(I - Pt) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        M   :: PageRankImPt{<:Number, <:Integer, T}, 
        v   :: Vector{T}
        ) where T <: AbstractFloat
    n = length(v)
    
    # compute pA'(diag(1/r^*))v
    copy!(out, v)
    out .*= M.inv_out_degrees
#    @views out[M.dangling_page] .= 0
    
    mul!(M.storage_n, transpose(M.A), out)
    copy!(out, M.storage_n)

#    out .*= M.inv_out_degrees
    out .*= M.telep
    
    mu_v = sum(v) / n
    out .+= (1 - M.telep) * mu_v
    copy!(M.storage_n, v)
    M.storage_n .*= M.dangling_page
    mu_had = sum(M.storage_n) / n
    out .+= M.telep * mu_had
    out .= v .- out
    
#    sleep(1e-2) # wait 10 ms as if your code takes 1ms
    return out
end

# overwrite `out` by `(I - P) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        Mt  :: Transpose{T, PageRankImPt{TA, IA, T}}, 
        v   :: Vector{T}
        ) where {TA<:Number, IA<:Integer, T <: AbstractFloat}
    M = Mt.parent
    n = length(v)
    copy!(out, v)
    mul!(M.storage_n, M.A, out)
    copy!(out, M.storage_n)
    out .*= M.inv_out_degrees
#    @views out[M.dangling_page] .= 0
    out .*= M.telep
    
    mu_v = sum(v) / n
    out .+= (1 - M.telep) * mu_v
    copy!(M.storage_n, M.dangling_page)
    M.storage_n .*= M.telep * mu_v
    out .+= M.storage_n
    out .= v .- out
#    sleep(1e-2) # wait 10 ms as if your code takes 1ms
    return out
end

In [12]:
M = PageRankImPt(A, 0.85)

916428×916428 PageRankImPt{Bool, Int64, Float64}:
  1.0         -1.63679e-7  -1.63679e-7  …  -1.63679e-7  -1.63679e-7
 -1.63679e-7   1.0         -1.63679e-7     -1.63679e-7  -1.63679e-7
 -1.63679e-7  -1.63679e-7   1.0            -1.63679e-7  -1.63679e-7
 -1.09119e-6  -1.09119e-6  -1.09119e-6     -1.09119e-6  -1.09119e-6
 -1.63679e-7  -1.63679e-7  -1.63679e-7     -1.63679e-7  -1.63679e-7
 -1.63679e-7  -1.63679e-7  -1.63679e-7  …  -1.63679e-7  -1.63679e-7
 -1.63679e-7  -1.63679e-7  -1.63679e-7     -1.63679e-7  -1.63679e-7
 -1.63679e-7  -1.63679e-7  -1.63679e-7     -1.63679e-7  -1.63679e-7
 -1.63679e-7  -1.63679e-7  -1.63679e-7     -1.63679e-7  -1.63679e-7
 -1.63679e-7  -1.63679e-7  -1.63679e-7     -1.63679e-7  -1.63679e-7
 -1.63679e-7  -1.63679e-7  -1.63679e-7  …  -1.63679e-7  -1.63679e-7
 -1.63679e-7  -1.63679e-7  -1.63679e-7     -1.63679e-7  -1.63679e-7
 -1.63679e-7  -1.63679e-7  -1.63679e-7     -1.63679e-7  -1.63679e-7
  ⋮                                     ⋱               
 -1.63679

To check correctness. Note 
$$
\mathbf{M}^T \mathbf{1} = \mathbf{0}
$$
and
$$
\mathbf{M} \mathbf{x} = \mathbf{0}
$$
for stationary distribution $\mathbf{x}$.

Download the solution file `pgrksol.csv.gz`. **Do not put this file in your Git**. You will lose points if you do. You can add a line `pgrksol.csv.gz` to your `.gitignore` file.

In [13]:
using CodecZlib, DelimitedFiles

isfile("pgrksol.csv.gz") || download("https://raw.githubusercontent.com/ucla-biostat-257/2022spring/master/hw/hw4/pgrksol.csv.gz")
xsol = open("pgrksol.csv.gz", "r") do io
    vec(readdlm(GzipDecompressorStream(io)))
end

916428-element Vector{Float64}:
 3.3783428216975054e-5
 2.0710155392568165e-6
 3.663065984832893e-6
 7.527510785028837e-7
 8.63328599674051e-7
 1.769418252415541e-6
 2.431230382883396e-7
 6.368417180141445e-7
 4.744973703681939e-7
 2.6895486110647536e-7
 3.18574314847409e-6
 7.375106374416742e-7
 2.431230382883396e-7
 ⋮
 1.1305006040148547e-6
 4.874825281822915e-6
 3.167946973112519e-6
 9.72688040308568e-7
 6.588614479285245e-7
 7.737011774300648e-7
 2.431230382883396e-7
 1.6219204214797293e-6
 3.912130060551738e-7
 2.431230382883396e-7
 7.296033831163157e-6
 6.330939996912478e-7

**You will lose all 35 points (Steps 1 and 2)** if the following statements throw AssertError.

In [14]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)

@assert norm(transpose(M) * ones(n)) < 1e-12

In [15]:
@assert norm(M * xsol) < 1e-12

### Step 2 (20 pts)

We want to benchmark the hot functions `mul!` to make sure they are efficient and allocate little memory.

In [16]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)
v, out = ones(n), zeros(n)
bm_mv = @benchmark mul!($out, $M, $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 90 samples with 1 evaluation.
 Range (min … max):  52.973 ms … 65.916 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     54.900 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   55.236 ms ±  1.738 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

        █ ▅▂▅▂▅▅▅  ▂█▅ ▂  █     ▅   ▂                          
  ▅▁▅▅█▅█▅███████▁████▁█▅▁█▅▅▅▁▅██████▁▁▁▁▅█▁▁▁▅█▁▁▁▁▅▅▁▁▁▁▁▅ ▁
  53 ms           Histogram: frequency by time          59 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [17]:
bm_mtv = @benchmark mul!($out, $(transpose(M)), $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 69 samples with 1 evaluation.
 Range (min … max):  69.505 ms … 80.942 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     72.220 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   72.365 ms ±  1.468 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

                 ▁▁  ▄█   ▄▁▁▁▄▁▄  ▁ ▄█  ▁▁▁▁     ▁            
  ▆▁▁▁▁▁▁▆▁▁▁▁▁▆▆██▆▆██▆▆▆███████▆▆█▆██▆▆████▁▁▆▆▆█▆▆▁▆▁▁▁▁▁▆ ▁
  69.5 ms         Histogram: frequency by time        74.8 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [18]:
clamp(10 - median(bm_mv).memory / 100, 0, 10) + 
clamp(10 - median(bm_mtv).memory / 100, 0, 10)

20.0

**Hint**: My median run times are 30-40 ms and memory allocations are 0 bytes.

### Step 3 (20 pts)

Let's first try to solve the PageRank problem by the GMRES method for solving linear equations. 

In [19]:
using KrylovKit

# normalize in-degrees to be the start point
x0   = vec(sum(A, dims = 1)) .+ 1.0
x0 ./= sum(x0)

# right hand side
b = zeros(n)

# warm up (compilation)
linsolve(M, b, x0, issymmetric = false, isposdef = false, maxiter = 1) 
# output is complex eigenvalue/eigenvector
(x_gmres, info), time_gmres, = @timed linsolve(M, b, x0, issymmetric = false, isposdef = false)

(value = ([3.378342822186503e-5, 2.071015539246995e-6, 3.6630659852345382e-6, 7.527510785597202e-7, 8.633285997155132e-7, 1.7694182527365293e-6, 2.431230382910348e-7, 6.368417180742664e-7, 4.744973703746486e-7, 2.689548611094569e-7  …  3.1679469739754828e-6, 9.726880410224357e-7, 6.588614478537671e-7, 7.737011774709108e-7, 2.431230382910348e-7, 1.6219204214234728e-6, 3.912130060628608e-7, 2.431230382910348e-7, 7.2960338313196555e-6, 6.330939996673809e-7], ConvergenceInfo: one converged value after 3 iterations and 72 applications of the linear map;
norms of residuals are given by (7.821150109559894e-13,).
), time = 8.004948483, bytes = 1009028732, gctime = 0.372343284, gcstats = Base.GC_Diff(1009028732, 137, 0, 75139, 4, 0, 372343284, 3, 2))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [20]:
@assert norm(x_gmres - xsol) < 1e-8

GMRES should be reasonably fast. The points you'll get is

In [21]:
clamp(20 / time_gmres * 20, 0, 20)

20.0

**Hint**: My runtime is about 7-8 seconds.

### Step 4 (20 pts)

Let's first try to solve the PageRank problem by the Arnoldi method for solving eigen problems. 

In [22]:
# warm up (compilation)
eigsolve(M, x0, 1, :SR, issymmetric = false, maxiter = 1)
# output is complex eigenvalue/eigenvector
(vals, vecs, info), time_arnoldi, = @timed eigsolve(M, x0, 1, :SR, issymmetric = false)

(value = (ComplexF64[-7.15218598150451e-16 + 0.0im], Vector{ComplexF64}[[0.005635826953807292 + 0.0im, 0.0003454914380782916 + 0.0im, 0.000611080849412509 + 0.0im, 0.00012557561626034896 + 0.0im, 0.0001440224053280302 + 0.0im, 0.0002951783050398427 + 0.0im, 4.055832828677737e-5 + 0.0im, 0.00010623935784851908 + 0.0im, 7.915671116195064e-5 + 0.0im, 4.486765066724741e-5 + 0.0im  …  0.0005284839899797011 + 0.0im, 0.00016226599147988602 + 0.0im, 0.00010991273837609453 + 0.0im, 0.00012907055855751432 + 0.0im, 4.055832828677737e-5 + 0.0im, 0.0002705723874320372 + 0.0im, 6.526302748333191e-5 + 0.0im, 4.055832828677737e-5 + 0.0im, 0.0012171406601327794 + 0.0im, 0.0001056141551069884 + 0.0im]], ConvergenceInfo: one converged value after 7 iterations and 99 applications of the linear map;
norms of residuals are given by (9.574560720282502e-14,).
), time = 13.208445288, bytes = 1581384773, gctime = 0.251071389, gcstats = Base.GC_Diff(1581384773, 213, 0, 77837, 20, 0, 251071389, 6, 0))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [23]:
@assert abs(Real(vals[1])) < 1e-8

In [24]:
x_arnoldi   = abs.(Real.(vecs[1]))
x_arnoldi ./= sum(x_arnoldi)
@assert norm(x_arnoldi - xsol) < 1e-8

Arnoldi should be reasonably fast. The points you'll get is

In [25]:
clamp(20 / time_arnoldi * 20, 0, 20)

20.0

**Hint**: My runtime is about 11-12 seconds.

## Q6 (5 pts) Results

List the top 20 pages you found and their corresponding PageRank score. Do they match the top 20 pages ranked according to in-degrees? 

### Answer

We compute the top 20 pages we've found using the GMRES and Arnoldi methods for solving eigen problems.

In [26]:
gmres_top_20 = Vector{Int64}(undef, 20)

for i in 1:20
    m, ix = findmax(x_gmres)
    println("Maximum ", i, "th ranking ", m, " at index ", ix, " for the GMRES method.")
    gmres_top_20[i] = ix
    deleteat!(x_gmres, ix)
end

Maximum 1th ranking 0.0009145812115800053 at index 597622 for the GMRES method.
Maximum 2th ranking 0.0009120131809576345 at index 41910 for the GMRES method.
Maximum 3th ranking 0.0008950559015956306 at index 163075 for the GMRES method.
Maximum 4th ranking 0.0008899344804879074 at index 537038 for the GMRES method.
Maximum 5th ranking 0.0007791031790941284 at index 384665 for the GMRES method.
Maximum 6th ranking 0.0007575423487418646 at index 504138 for the GMRES method.
Maximum 7th ranking 0.0007177642925385278 at index 486978 for the GMRES method.
Maximum 8th ranking 0.0007108483954766748 at index 605850 for the GMRES method.
Maximum 9th ranking 0.0007055182683476246 at index 32164 for the GMRES method.
Maximum 10th ranking 0.0007021658710779174 at index 558785 for the GMRES method.
Maximum 11th ranking 0.0006950751256673672 at index 551823 for the GMRES method.
Maximum 12th ranking 0.0006762276023217714 at index 765324 for the GMRES method.
Maximum 13th ranking 0.0006546558408533

In [27]:
arnoldi_top_20 = Vector{Int64}(undef, 20)

for i in 1:20
    m, ix = findmax(x_arnoldi)
    println("Maximum ", i, "th ranking ", m, " at index ", ix, " for the Arnoldi method.")
    arnoldi_top_20[i] = ix
    deleteat!(x_arnoldi, ix)
end

Maximum 1th ranking 0.0009145812114518429 at index 597622 for the Arnoldi method.
Maximum 2th ranking 0.0009120131809986378 at index 41910 for the Arnoldi method.
Maximum 3th ranking 0.0008950559016075283 at index 163075 for the Arnoldi method.
Maximum 4th ranking 0.000889934480441449 at index 537038 for the Arnoldi method.
Maximum 5th ranking 0.000779103179016405 at index 384665 for the Arnoldi method.
Maximum 6th ranking 0.0007575423485846912 at index 504138 for the Arnoldi method.
Maximum 7th ranking 0.0007177642925774892 at index 486978 for the Arnoldi method.
Maximum 8th ranking 0.0007108483954459599 at index 605850 for the Arnoldi method.
Maximum 9th ranking 0.0007055182681589602 at index 32164 for the Arnoldi method.
Maximum 10th ranking 0.0007021658710708973 at index 558785 for the Arnoldi method.
Maximum 11th ranking 0.0006950751256446756 at index 551823 for the Arnoldi method.
Maximum 12th ranking 0.0006762276025731853 at index 765324 for the Arnoldi method.
Maximum 13th rank

In [28]:
gmres_top_20 == arnoldi_top_20

true

In [29]:
gmres_top_20 == in_degree_top_20

false

While the top 20 pages match for both GMRES and Arnoldi method, they do not match those for the in-degree counts.

## Q7 Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on a network of one million nodes.